# Stores

> **Note:** <br> 
> You can find the official example at [https://docs.geoserver.org/2.25.x/en/user/rest/stores.html](https://docs.geoserver.org/2.25.x/en/user/rest/stores.html)

## Setup

### Imports

First, we need to import the necessary modules and classes.

In [1]:
from pathlib import Path
from geoserver import GeoServer
from geoserver.exceptions import GeoServerError

### GeoServer Connection

Connect to the running GeoServer instance and create a workspace and a store.

In [2]:
# Setup the geoserver instance
geoserver = GeoServer(
    service_url="http://localhost:8080/geoserver",
    username="admin",
    password="geoserver",
)

Clean up the workspace and store after running the examples.

In [3]:
if geoserver.workspace_exists("demo"):
    geoserver.delete_workspace("demo", recurse=True)

geoserver.create_workspace_from_name("demo")

'Created'

### Config

We'll set up the configuration for the notebook:

In [4]:
# Directory containing sample data
EXAMPLES_DIR = Path("../examples")
assert EXAMPLES_DIR.exists(), f"The directory {EXAMPLES_DIR} does not exist."

## Uploading a shapefile

Create a new store `roads` by uploading a shapefile `roads.zip`.

In [5]:
file_path = EXAMPLES_DIR / "vectors" / "roads.zip"
assert file_path.exists(), f"The file {file_path.as_posix()!r} does not exist."


geoserver.upload_data_store(file=file_path, workspace="demo")

'Created'

## Retrieving a store

Retrieve information about a specific store. You can specify the response format as JSON or XML using the `format` parameter.

In [6]:
geoserver.get_data_store(workspace="demo", store="roads")

{'dataStore': {'name': 'roads',
  'type': 'Shapefile',
  'enabled': True,
  'workspace': {'name': 'demo',
   'href': 'http://localhost:8080/geoserver/rest/workspaces/demo.json'},
  'connectionParameters': {'entry': [{'@key': 'namespace', '$': 'http://demo'},
    {'@key': 'url', '$': 'file:/opt/geoserver/data_dir/data/demo/roads/'}]},
  '_default': False,
  'dateCreated': '2024-06-05 22:59:52.539 UTC',
  'disableOnConnFailure': False,
  'featureTypes': 'http://localhost:8080/geoserver/rest/workspaces/demo/datastores/roads/featuretypes.json'}}

In [7]:
xml = geoserver.get_data_store(workspace="demo", store="roads", format="xml")
print(xml)

<dataStore>
  <name>roads</name>
  <type>Shapefile</type>
  <enabled>true</enabled>
  <workspace>
    <name>demo</name>
    <atom:link xmlns:atom="http://www.w3.org/2005/Atom" rel="alternate" href="http://localhost:8080/geoserver/rest/workspaces/demo.xml" type="application/xml"/>
  </workspace>
  <connectionParameters>
    <entry key="namespace">http://demo</entry>
    <entry key="url">file:/opt/geoserver/data_dir/data/demo/roads/</entry>
  </connectionParameters>
  <__default>false</__default>
  <dateCreated>2024-06-05 22:59:52.539 UTC</dateCreated>
  <disableOnConnFailure>false</disableOnConnFailure>
  <featureTypes>
    <atom:link xmlns:atom="http://www.w3.org/2005/Atom" rel="alternate" href="http://localhost:8080/geoserver/rest/workspaces/demo/datastores/roads/featuretypes.xml" type="application/xml"/>
  </featureTypes>
</dataStore>


## Listing featuretype details

By default when a shapefile is uploaded, a featuretype is automatically created. This is true only if you use the `upload_data_store` method. If you use the `create_data_store` method, you will have to create the featuretype manually.

In [8]:
geoserver.get_feature_type(workspace="demo", featuretype="roads")

{'featureType': {'name': 'roads',
  'nativeName': 'roads',
  'namespace': {'name': 'demo',
   'href': 'http://localhost:8080/geoserver/rest/namespaces/demo.json'},
  'title': 'roads',
  'keywords': {'string': ['features', 'roads']},
  'srs': 'EPSG:404000',
  'nativeBoundingBox': {'minx': 971280.0291221511,
   'maxx': 1009968.0258890537,
   'miny': 188493.74372751566,
   'maxy': 259547.82405279655},
  'latLonBoundingBox': {'minx': 971280.0291221511,
   'maxx': 1009968.0258890537,
   'miny': 188493.74372751566,
   'maxy': 259547.82405279655,
   'crs': 'EPSG:4326'},
  'projectionPolicy': 'FORCE_DECLARED',
  'enabled': True,
  'store': {'@class': 'dataStore',
   'name': 'demo:roads',
   'href': 'http://localhost:8080/geoserver/rest/workspaces/demo/datastores/roads.json'},
  'serviceConfiguration': False,
  'simpleConversionEnabled': False,
  'maxFeatures': 0,
  'numDecimals': 0,
  'padWithZeros': False,
  'forcedDecimal': False,
  'overridingServiceSRS': False,
  'skipNumberMatched': False

## Adding an existing shapefile

Publish a shapefile that already exists on the server without needing to be uploaded. We will use the `roads` store that we created earlier.

```python
geoserver.upload_data_store(
    file="file:/opt/geoserver/data_dir/data/demo/roads/roads.shp", 
    workspace="demo", 
    store="roads2",
    filename="roads2.shp",
)
```

## Adding a directory of existing shapefiles

Create a store containing a directory of shapefiles that already exists on the server without needing to be uploaded.

```python
geoserver.upload_data_store(
    file="file:/opt/geoserver/data_dir/data/demo", 
    workspace="demo", 
    store="roads2",
    configure="all"
)
```

## Adding a PostGIS database store

Add an existing PostGIS database as a new store. The connection parameters available from the `docker-compose.yml` file are:

- host: `postgis`
- port: `5432`
- database: `db`
- user: `admin`
- password: `postgres`
- dbtype: `postgis`

> **Note:** <br>
> This example assumes that a PostGIS database named `db` is present on the local system and is accessible by the user `admin`.

In [9]:
# Using JSON format
body = {
    "dataStore": {
        "name": "db",
        "description": "PostGIS connection",
        "connectionParameters": {
            "host": "postgis",
            "port": "5432",
            "database": "db",
            "user": "admin",
            "passwd": "postgres",
            "dbtype": "postgis",
            "schema": "public",
            "Expose primary keys": "true",
            "Loose bbox": "true",
            "Estimated extends": "true",
            "fetch size": "1000",
            "Max open prepared statements": "50",
            "preparedStatements": "false",
            "validate connections": "true",
            "validate connections on borrow": "true",
            "validate connections on return": "true",
            "Connection timeout": "20",
            "Eviction run periodicity": "3600",
            "Min evictable idle time": "300",
            "Max active": "50",
            "Max idle": "10",
            "Max wait": "10000",
            "Test on borrow": "true",
            "Test while idle": "true",
            "Time between eviction runs": "60000",
        }
    }
}

# Using XML format
body = """
<dataStore>
    <name>db</name>
    <description>PostGIS database</description>
    <connectionParameters>
        <entry key="host">postgis</entry>
        <entry key="port">5432</entry>
        <entry key="database">db</entry>
        <entry key="user">admin</entry>
        <entry key="passwd">postgres</entry>
        <entry key="dbtype">postgis</entry>
        <entry key="schema">public</entry>
        <entry key="Expose primary keys">true</entry>
        <entry key="Loose bbox">true</entry>
        <entry key="Estimated extends">true</entry>
        <entry key="fetch size">1000</entry>
        <entry key="Max open prepared statements">50</entry>
        <entry key="preparedStatements">false</entry>
        <entry key="validate connections">true</entry>
        <entry key="validate connections on borrow">true</entry>
        <entry key="validate connections on return">true</entry>
        <entry key="Connection timeout">20</entry>
        <entry key="Eviction run periodicity">3600</entry>
        <entry key="Min evictable idle time">300</entry>
        <entry key="Max active">50</entry>
        <entry key="Max idle">10</entry>
        <entry key="Max wait">10000</entry>
        <entry key="Test on borrow">true</entry>
        <entry key="Test while idle">true</entry>
        <entry key="Time between eviction runs">60000</entry>
    </connectionParameters>
</dataStore>
"""


geoserver.create_data_store(workspace="demo", body=body)

'Created'

## Listing a PostGIS database store details

Retrieve information about a PostGIS store.

In [10]:
xml = geoserver.get_data_store(workspace="demo", store="db", format="xml")
print(xml)

<dataStore>
  <name>db</name>
  <description>PostGIS database</description>
  <type>PostGIS</type>
  <enabled>true</enabled>
  <workspace>
    <name>demo</name>
    <atom:link xmlns:atom="http://www.w3.org/2005/Atom" rel="alternate" href="http://localhost:8080/geoserver/rest/workspaces/demo.xml" type="application/xml"/>
  </workspace>
  <connectionParameters>
    <entry key="schema">public</entry>
    <entry key="Max idle">10</entry>
    <entry key="Max open prepared statements">50</entry>
    <entry key="preparedStatements">false</entry>
    <entry key="database">db</entry>
    <entry key="validate connections on borrow">true</entry>
    <entry key="host">postgis</entry>
    <entry key="Loose bbox">true</entry>
    <entry key="Min evictable idle time">300</entry>
    <entry key="Estimated extends">true</entry>
    <entry key="fetch size">1000</entry>
    <entry key="Eviction run periodicity">3600</entry>
    <entry key="Expose primary keys">true</entry>
    <entry key="validate connec

## Publishing a table from an existing PostGIS store

Publish a new featuretype from a PostGIS store table `buildings`.

> **Note:** <br>
> This example assumes the table has already been created. You can create the table using the following command:
> ```bash
> ogr2ogr -f PostgreSQL PG:"host=localhost port=5432 user=admin dbname=db password=postgres" examples/vectors/jamoat-db.shp -nlt PROMOTE_TO_MULTI -lco OVERWRITE=YES
> ```

In [11]:
!ogr2ogr -f PostgreSQL PG:"host=localhost port=5432 user=admin dbname=db password=postgres" ../examples/vectors/jamoat-db.shp -nlt PROMOTE_TO_MULTI -lco OVERWRITE=YES

In [12]:
# Using JSON format
body = {
    "featureType": {
        "name": "jamoat_db",
        "title": "jamoat_db",
        "advertised": "true",
    }
}

# Using XML format
body = """
<featureType>
    <name>jamoat_db</name>
    <title>jamoat_db</title>
    <advertised>true</advertised>
</featureType>
"""


geoserver.create_feature_type(workspace="demo", store="db", body=body)

'Created'

> **Note:** <br>
> This layer can viewed with a WMS GetMap request at the following URL:
> ```
> http://localhost:8080/geoserver/wms/reflect?layers=demo:jamoat_db
> ```

## Creating a PostGIS table

Create a new featuretype in GeoServer and simultaneously create a table in PostGIS.

> **Note:** <br>
> The `db` store must be a PostGIS store for this to succeed.

In [13]:
# Using XML format
body = """
<featureType>
  <name>annotations</name>
  <nativeName>annotations</nativeName>
  <title>Annotations</title>
  <srs>EPSG:4326</srs>
  <attributes>
    <attribute>
      <name>the_geom</name>
      <binding>org.locationtech.jts.geom.Point</binding>
    </attribute>
    <attribute>
      <name>description</name>
      <binding>java.lang.String</binding>
    </attribute>
    <attribute>
      <name>timestamp</name>
      <binding>java.util.Date</binding>
    </attribute>
  </attributes>
</featureType>
"""


try:
  geoserver.create_feature_type(workspace="demo", store="db", body=body)
except GeoServerError as e:
  if e.status_code == 500:
    print(f"[WARNING] Could not add featuretype: {e}")
    print("[WARNING] It is possible that the featuretype already exists in the database.")

A new and empty table named `annotations` in the `db` database will be created as well.

## Adding an external WMTS Store

Create a new WMTS store `Basemap-Nat-Geo-Datastore`.

In [14]:
# Using JSON format
body = {
    "wmtsStore": {
        "name": "basemap-nat-geo-datastore",
        "description": "esri-street-map",
        "capabilitiesURL": "https://services.arcgisonline.com/arcgis/rest/services/NatGeo_World_Map/MapServer/WMTS/1.0.0/WMTSCapabilities.xml",
        "type": "WMTS",
    }
}

# Using XML format
body = """
<wmtsStore>
  <name>basemap-nat-geo-datastore</name>
  <description>esri-street-map</description>
  <capabilitiesURL>https://services.arcgisonline.com/arcgis/rest/services/NatGeo_World_Map/MapServer/WMTS/1.0.0/WMTSCapabilities.xml</capabilitiesURL>
  <type>WMTS</type>
</wmtsStore>
"""


geoserver.create_wmts_store(workspace="demo", body=body)

'Created'

In [15]:
geoserver.get_wmts_layers(workspace="demo")

{'layers': {'layer': [{'name': 'annotations',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/demo/layers/annotations.json'},
   {'name': 'jamoat_db',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/demo/layers/jamoat_db.json'},
   {'name': 'roads',
    'href': 'http://localhost:8080/geoserver/rest/workspaces/demo/layers/roads.json'}]}}

## Adding an external WMTS Layer

Publish a new WMTS Layer `NatGeo_World_Map` from the WMTS store `Basemap-Nat-Geo-Datastore`.

In [16]:
# Using JSON format
body = {
    "wmtsLayer": {
        "name": "NatGeo_World_Map",
    }
}

# Using XML format
body = """
<wmtsLayer>
  <name>NatGeo_World_Map</name>
</wmtsLayer>
"""


geoserver.create_wmts_layer(workspace="demo", store="basemap-nat-geo-datastore", body=body)

'Created'